In [ ]:
import os
import re
import numpy as np
import rasterio
from rasterio.transform import from_origin

In [ ]:
# Paths to the folders
grid_folder = "./grid"
ndvi_folder = "./NDVI2"
original_metadata_folder = "./Original"
output_base_folder = "./output_NDVI"
os.makedirs(output_base_folder, exist_ok=True)

# Years to process
years = range(1988, 2022)  # 1988 to 2020

# Function to find the exact matching original TIFF file for each area
def find_original_file(area_name):
    for filename in os.listdir(original_metadata_folder):
        # Match area name anywhere after the date prefix
        if re.search(rf"{area_name}\.tif$", filename):
            return os.path.join(original_metadata_folder, filename)
    return None

# Loop through each grid folder (area of interest)
for area_name in os.listdir(ndvi_folder):
    ndvi_area_folder = os.path.join(ndvi_folder, area_name)

    # Ensure the folder exists
    if not os.path.isdir(ndvi_area_folder):
        continue

    # Match the original TIFF file for this area
    original_file = find_original_file(area_name)
    if original_file is None:
        print(f"No original TIFF file found for {area_name}. Skipping.")
        continue

    # Extract metadata from the corresponding original TIFF file
    with rasterio.open(original_file) as original_src:
        pixel_size_x = original_src.transform.a
        pixel_size_y = original_src.transform.e  # Already negative for north-down
        origin_x = original_src.transform.c
        origin_y = original_src.transform.f
        raster_width = original_src.width
        raster_height = original_src.height
        crs = original_src.crs  # Coordinate reference system
        nodata_value = original_src.nodata if original_src.nodata is not None else np.nan

        # Print metadata for debugging
        print(f"Original File: {original_file}")
        print(f"  Origin: ({origin_x}, {origin_y})")
        print(f"  Pixel Size: ({pixel_size_x}, {pixel_size_y})")
        print(f"  Dimensions: {raster_width}x{raster_height}")
        print(f"  CRS: {crs}")

    # Create output folder for this area
    output_area_folder = os.path.join(output_base_folder, area_name)
    os.makedirs(output_area_folder, exist_ok=True)

    # Loop through each year
    for year in years:
        ndvi_file = os.path.join(ndvi_area_folder, f"{year}_{area_name}.tif")

        # Check if the NDVI file exists
        if not os.path.isfile(ndvi_file):
            print(f"NDVI file for {year} in {area_name} not found. Skipping.")
            continue

        # Open the NDVI raster
        with rasterio.open(ndvi_file) as src:
            # Use the transform and metadata from the original file
            transform = from_origin(origin_x, origin_y, pixel_size_x, -pixel_size_y)

            # Create the output raster grid
            ndvi_grid = np.full((raster_height, raster_width), nodata_value, dtype=np.float32)

            # Loop through each row and column
            for row in range(raster_height):
                for col in range(raster_width):
                    # Calculate the pixel center coordinates
                    x, y = rasterio.transform.xy(transform, row, col, offset="center")
                    try:
                        # Sample NDVI value at pixel center
                        ndvi_value = list(src.sample([(x, y)]))[0][0]
                        if ndvi_value is not None:
                            ndvi_grid[row, col] = ndvi_value
                    except IndexError:
                        # Handle out-of-bounds or invalid pixels
                        pass

            # Save the NDVI grid as a new GeoTIFF
            output_tif_path = os.path.join(output_area_folder, f"{year}_{area_name}.tif")
            with rasterio.open(
                output_tif_path,
                "w",
                driver="GTiff",
                height=raster_height,
                width=raster_width,
                count=1,
                dtype="float32",  # Ensure Float32 data type
                crs=crs,  # Keep CRS from the original file
                transform=transform,
                nodata=nodata_value,
            ) as dst:
                dst.write(ndvi_grid, 1)
                print(f"Saved raster: {output_tif_path}")